In [1]:
# import os
# os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_spouse'

# from snorkel import SnorkelSession
# session = SnorkelSession()

In [2]:
# import os

# from snorkel.models import candidate_subclass
# from snorkel.contrib.babble import ExplanationIO
# from snorkel.contrib.babble.pipelines import config
# from tutorials.babble.spouse import SpousePipeline

# config = configuration
# config['domain'] = 'spouse'
# config['splits'] = [0,1,2]
# config['babbler_split'] = 0
# config['max_docs'] = None
# config['parallelism'] = 1
# config['traditional'] = False
# config['majority_vote'] = True
# config['verbose'] = True
# config['display_marginals'] = True
# config['display_accuracies'] = True
# config['display_learned_accuracies'] = True

# Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

# sm = SpousePipeline(session, Spouse, config)

In [3]:
# %time sm.parse(config=config)

In [4]:
# %time sm.extract(config=config)

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/babble/data/')
os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_spouse'
# os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_bike'

from snorkel import SnorkelSession
session = SnorkelSession()

In [6]:
from snorkel.models import candidate_subclass
from snorkel.contrib.babble import ExplanationIO
from mturk_processing import MTurkHelper

import os

# SPOUSE
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
candidate_class = Spouse
mode = 'text'
# fpath = os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/grad_explanations.tsv'
fpath = os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/mturk_explanations_all.tsv'
expio = ExplanationIO()
explanations = expio.read(fpath)

# BIKER
# Biker = candidate_subclass('Biker', ['person', 'bike'])
# candidate_class = Biker
# mode = 'image'
# helper = MTurkHelper(candidates, num_hits=40, workers_per_hit=3)
# output_csv_path= os.environ['SNORKELHOME'] + '/tutorials/babble/bike/data/VisualGenome_all_out.csv'
# explanations = helper.postprocess_visual(output_csv_path, verbose=False)

Read 445 explanations from /Users/bradenjh/repo...ata/mturk_explanations_all.tsv


In [7]:
candidates = session.query(candidate_class).filter(candidate_class.split == 0).all()
print(len(candidates))

23490


In [8]:
from snorkel.contrib.babble import link_explanation_candidates

explanations = link_explanation_candidates(explanations, candidates)

Building list of target candidate ids...
Collected 155 unique target candidate ids from 445 explanations.
Gathering desired candidates...
Found 155/155 desired candidates
Linking explanations to candidates...
Linked 445/445 explanations


In [9]:
# for exp in explanations:
#     if exp.condition == 'widow is mentioned between the two names':
#         explanations = [exp]
#         break
# print(explanations[0])

In [10]:
exp_iterator = iter(explanations)

In [11]:
exp = exp_iterator.next()
from snorkel.viewer import SentenceNgramViewer
print(exp.condition)
sv = SentenceNgramViewer([exp.candidate], session, n_per_page=3, height=150)
sv

Politicians


<IPython.core.display.Javascript object>

In [12]:
# exp.condition = 'since his widow is mentioned between the two names'

In [13]:
# explanations = [exp]
# explanations

In [14]:
from snorkel.contrib.babble import Babbler
user_lists = {}
babbler = Babbler(mode=mode, 
                  candidate_class=candidate_class, 
                  explanations=explanations,
                  string_format='implicit')

Created grammar with 473 rules


In [15]:
# for exp in explanations:
#     LF_dict = babbler.semparser.parse_and_evaluate(exp, show_erroring=True)
#     num_correct = len(LF_dict['correct'])
#     num_passing = len(LF_dict['passing'])
#     num_acceptable = num_correct + num_passing
#     if not num_acceptable > 0:
#         print(LF_dict)
#         babbler.semparser.grammar.print_chart()
#         parses = babbler.semparser.parse(exp, return_parses=True)
#         import pdb; pdb.set_trace()
#     else:
#         print('Acceptable: {}'.format(num_acceptable))

In [16]:
# for exp in explanations[:5]: print(exp.condition)

In [17]:
# from snorkel.contrib.babble import Explanation
# exp = Explanation("person1 equals person2", True, name='test_exp')
# homemades = [exp]

In [18]:
lfs = babbler.generate_lfs()

350 parses created from 260 out of 445 explanation(s)
Parsed 350 LFs from 445 explanations.


In [19]:
babbler.filter_duplicate_semantics()

Filtered to 260 LFs with duplicate semantics filter (90 filtered).


In [20]:
babbler.filter_consistency()

Filtered to 101 LFs with consistency filter (159 filtered).


In [27]:
babbler.get_parses(translate=True)

["return 1 if call(contains('tied the knot'), text(sentence())) else 0",
 "return 1 if call(contains('husband'), text(sentence())) else 0",
 "return -1 if call(in text(between([arg1,arg2])), 'brother') else 0",
 "return -1 if call(contains('gangsters'), text(sentence())) else 0",
 "return 1 if call(contains('wife'), text(sentence())) else 0",
 "return 1 if call(contains('widow'), text(sentence())) else 0",
 "return 1 if call(contains('son'), text(sentence())) else 0",
 "return 1 if call(contains('proposal'), text(sentence())) else 0",
 "return -1 if call(in text(left(arg1,'.leq',3,'words')), 'daughter') else 0",
 "return -1 if call(in text(left(arg2,'.eq',1,'words')), 'beau') else 0",
 "return 1 if call(in text(between([arg1,arg2])), 'married') else 0",
 "return 1 if all(map(in text(sentence()), ['marriage',text(arg1),text(arg2)])) else 0",
 "return 1 if call(in text(between([arg1,arg2])), 'widow') else 0",
 "return 1 if call(contains('her husband'), text(sentence())) else 0",
 "return

In [26]:
babbler.get_explanations()

[Explanation("Explanation12: True, It says they "tied the knot""),
 Explanation("Explanation123: True, It refers to Briath as Jodi's "husband""),
 Explanation("Explanation153: False, brother is between the names"),
 Explanation("Explanation164: False, it says they are gangsters"),
 Explanation("Explanation17: True, the text refers to person2 as being the wife of person1"),
 Explanation("Explanation172: True, the text refers to person2 as the widow of person1"),
 Explanation("Explanation210: True, text states that person1 and person2 have a son and they have the same last name"),
 Explanation("Explanation234: True, the text mentions the proposal from person1 to person2"),
 Explanation("Explanation241: False, the word "daughter" occurs within three words to the left of person1."),
 Explanation("Explanation250: False, "beau" is immediately before person 2"),
 Explanation("Explanation261: True, the word married is between the names"),
 Explanation("Explanation262: True, Article has marriag

In [23]:
%time babbler.generate_label_matrix(split=1, parallelism=1)

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 1min 9s, sys: 4.78 s, total: 1min 14s
Wall time: 2min 10s


<2456x101 sparse matrix of type '<type 'numpy.int64'>'
	with 186647 stored elements in Compressed Sparse Row format>

In [24]:
babbler.filter_uniform_signatures()

Filtered to 54 LFs with uniform signatures filter (47 filtered).


In [25]:
babbler.filter_duplicate_signatures()

Filtered to 21 LFs with duplicate signatures filter (33 filtered).


In [ ]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

In [ ]:
babbler.label_matrix.lf_stats(session, labels=L_gold_dev)